In [2]:
from sqlalchemy import create_engine
import pandas as pd

user = 'root'  
password = 'password'  
host = 'localhost'  
database = 'sakila'

engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

with engine.connect() as connection:
    print("Conexión exitosa")

Conexión exitosa


In [3]:
def rentals_month(engine, month, year):
    query = f"""
    SELECT customer_id, COUNT(*) AS rental_count
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    GROUP BY customer_id;
    """
    # Ejecutar la consulta y devolver como DataFrame
    return pd.read_sql(query, engine)

# Prueba la función
df_may = rentals_month(engine, 5, 2005)
print(df_may.head())

   customer_id  rental_count
0            1             2
1            2             1
2            3             2
3            5             3
4            6             3


In [4]:
def rental_count_month(df, month, year):
    column_name = f'rentals_{str(month).zfill(2)}_{year}'
    df.rename(columns={'rental_count': column_name}, inplace=True)
    return df

# Prueba la función
df_may = rental_count_month(df_may, 5, 2005)
print(df_may.head())

   customer_id  rentals_05_2005
0            1                2
1            2                1
2            3                2
3            5                3
4            6                3


In [5]:
def compare_rentals(df1, df2):
    # Unir DataFrames en el campo 'customer_id'
    combined_df = pd.merge(df1, df2, on='customer_id')
    # Calcular la diferencia
    col_names = combined_df.columns[1:]
    combined_df['difference'] = combined_df[col_names[0]] - combined_df[col_names[1]]
    return combined_df

# Prueba la función
df_june = rentals_month(engine, 6, 2005)
df_june = rental_count_month(df_june, 6, 2005)
result = compare_rentals(df_may, df_june)
print(result.head())

   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1                2                7          -5
1            2                1                1           0
2            3                2                4          -2
3            5                3                5          -2
4            6                3                4          -1
